In [7]:
import dotenv, os, torch
from pinecone import Pinecone
from sentence_transformers import SentenceTransformer
from main import query_pinecone_from_embedding, query_controller, merge_audio, get_topics_from_ids
import pandas as pd

device = "cuda" if torch.cuda.is_available() else "cpu"
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"), environment="production")
pc_index = pc.Index("hackathon24s")
model = SentenceTransformer("Snowflake/snowflake-arctic-embed-l").to(device)


You try to use a model that was created with version 2.7.0.dev0, however, your version is 2.7.0. This might cause unexpected behavior or errors. In that case, try to update to the latest version.





In [8]:
## load db
import sqlite3
import csv

# Connect to the SQLite database
conn = sqlite3.connect('db.sqlite3')
cursor = conn.cursor()


# Query the database for data
cursor.execute("SELECT * FROM main_clip")
data = cursor.fetchall()

# Write data to CSV
with open('output_filename.csv', 'w', newline='') as file:
    csv_writer = csv.writer(file)
    csv_writer.writerow([i[0] for i in cursor.description])  # write headers
    csv_writer.writerows(data)

# Close the connection
conn.close()

In [27]:
embedding = query_controller("I'd like to learn more about Google and Gemini", model)
pc_response = query_pinecone_from_embedding(embedding, pc_index)
ids = [str(i[0]) for i in pc_response]
thumbnail = pc_response[0][2]
topics = get_topics_from_ids(ids)
custom_pod_s3_url = await merge_audio(ids)

## This is the final output to the frontend
return_item = {
    "thumbnail": thumbnail,
    "topics": topics,
    "url": custom_pod_s3_url
}

['security', 'politics', 'politics', nan, 'artificial intelligence']


FileNotFoundError: [Errno 2] No such file or directory: 'audio/merged/128_merge.wav'